<a href="https://colab.research.google.com/github/AntonioCuba123/skills-introduction-to-github/blob/main/Ejercicio_de_Predicci%C3%B3n_Pron%C3%B3stico_de_Ventas_Totales_Diarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Ejercicio de Predicción: Pronóstico de Ventas Totales Diarias (Datos Ficticios Incluidos)

# --- 1. Librerías y Parámetros ---
import pandas as pd
import numpy as np
from datetime import date, timedelta
import random
from prophet import Prophet # Importa la clase Prophet
import matplotlib.pyplot as plt # Para visualización
import seaborn as sns # Para estilo de gráficos

# Configuración de estilo para los gráficos
sns.set_style("darkgrid")


# --- 2. Parámetros de Simulación de Datos Ficticios ---
dias_historicos_pred = 3 * 365 # 3 años de datos diarios

# Definir el rango de fechas históricas (hasta ayer desde la fecha actual simulada)
fecha_fin_hist = date.today() - timedelta(days=1)
fecha_inicio_hist = fecha_fin_hist - timedelta(days=dias_historicos_pred - 1)

# Parámetros de ventas diarias totales (simuladas en Soles)
base_ventas_dia_tienda = 5000 # Venta base promedio diaria en Soles
ruido_ventas_tienda = 1500 # Variación aleatoria típica

# Factores por día de la semana (ej: fines de semana más ventas)
factor_dia_semana_pred = {
    0: 0.8,  # Lunes (más bajo)
    1: 0.9,  # Martes
    2: 1.0,  # Miércoles
    3: 1.1,  # Jueves
    4: 1.3,  # Viernes
    5: 1.6,  # Sábado (pico)
    6: 1.4   # Domingo
}

# Simular estacionalidad anual (ej: más ventas en campañas como Día de la Madre, Navidad, Fiestas Patrias - Julio)
def factor_estacional_anual_pred(fecha):
    dia_del_año = fecha.timetuple().tm_yday
    # Picos inventados: Día de la Madre (Mayo ~130), Fiestas Patrias (Julio ~205), Navidad/Año Nuevo (Dic ~355)
    picos_dias = [130, 205, 355]
    factores_pico = [0.25, 0.15, 0.40] # Intensidad de los picos (ej: 25% más en Mayo, 40% más en Dic)
    suavizado = 10 # Días de influencia del pico

    estacionalidad = 1.0
    for i, pico_dia in enumerate(picos_dias):
        distancia_al_pico = min(abs(dia_del_año - pico_dia), 365 - abs(dia_del_año - pico_dia)) # Distancia circular en el año
        if distancia_al_pico < suavizado:
            estacionalidad += factores_pico[i] * (1 - distancia_al_pico / suavizado) # El efecto disminuye con la distancia

    # También añadir estacionalidad más suave (ej: mayor venta en el semestre 2)
    semestre_factor = 1 + 0.1 * (fecha.month - 6.5) / 6.5 # Ligeramente más bajo en Ene-Jun, más alto en Jul-Dic

    return estacionalidad * semestre_factor

# Simular una tendencia ligera al alza a lo largo de los 3 años
def factor_tendencia(fecha, fecha_inicio, dias_totales):
    # Convert fecha_inicio to Timestamp for consistent types
    fecha_inicio = pd.Timestamp(fecha_inicio)
    dias_transcurridos = (fecha - fecha_inicio).days
    progreso = dias_transcurridos / (dias_totales - 1) # 0 al inicio, 1 al final
    return 1 + 0.15 * progreso # Simula un crecimiento total del 15% en 3 años


# --- 3. Generar Datos Históricos Ficticios ---
print("--- Generando Datos Históricos de Ventas Diarias Ficticios ---")
datos_ventas_diarias = []
fechas = pd.date_range(start=fecha_inicio_hist, end=fecha_fin_hist, freq='D')

np.random.seed(47) # Semilla para reproducibilidad

for fecha in fechas:
    # Calcular ventas base con tendencia y estacionalidad
    ventas_base = base_ventas_dia_tienda * factor_tendencia(fecha, fecha_inicio_hist, dias_historicos_pred) * factor_estacional_anual_pred(fecha)

    # Añadir ruido diario aleatorio
    ruido = np.random.normal(0, ruido_ventas_tienda)

    # Aplicar factor del día de la semana
    factor_ds = factor_dia_semana_pred[fecha.weekday()]

    # Calcular ventas simuladas
    ventas_simuladas = (ventas_base + ruido) * factor_ds

    # Asegurarse de que las ventas no sean negativas y redondear a entero (para Soles)
    ventas_finales = max(0, round(ventas_simuladas))

    datos_ventas_diarias.append([fecha, ventas_finales])

# Crear el DataFrame directamente en memoria
df_ventas_diarias = pd.DataFrame(datos_ventas_diarias, columns=['Fecha', 'Ventas_Soles'])

print(f"DataFrame creado con {len(df_ventas_diarias)} filas.")
print("Primeras 5 filas:")
print(df_ventas_diarias.head())


# --- 4. Preparar Datos para Prophet y Entrenar Modelo ---
print("\n--- Preparando Datos y Entrenando Modelo Prophet ---")

# Prophet necesita las columnas nombradas 'ds' (fecha) y 'y' (valor a pronosticar)
df_prophet_ventas = df_ventas_diarias.rename(columns={'Fecha': 'ds', 'Ventas_Soles': 'y'})

# Crear y entrenar el modelo Prophet
model_ventas = Prophet(
    seasonality_mode='additive', # O 'multiplicative' si la estacionalidad aumenta con la tendencia
    yearly_seasonality=True,    # Capturar estacionalidad anual
    weekly_seasonality=True,    # Capturar estacionalidad semanal
    daily_seasonality=False     # No es necesario para datos diarios agregados
)
# Puedes añadir días festivos si tienes una lista, Prophet los considera como puntos atípicos o patrones
# model_ventas.add_country_holidays(country_name='PE') # Intentar añadir días festivos de Perú si están disponibles en Prophet

model_ventas.fit(df_prophet_ventas)
print("Modelo Prophet entrenado.")


# --- 5. Crear Fechas Futuras y Hacer la Predicción ---
print("\n--- Creando Fechas Futuras y Haciendo la Predicción ---")

# Crear un DataFrame con las fechas para las cuales queremos predecir (próximos 30 días)
# include_history=False asegura que solo se generen las fechas futuras
dias_pronostico_pred = 30
future_ventas = model_ventas.make_future_dataframe(periods=dias_pronostico_pred, include_history=False)

# Usar el modelo entrenado para predecir las ventas para las fechas futuras
forecast_ventas = model_ventas.predict(future_ventas)
print("Predicción generada.")


# --- 6. Mostrar Resultados del Pronóstico ---
print("\n--- Pronóstico de Ventas Totales Diarias (Soles) para los Próximos 30 Días ---")

# Seleccionar las columnas relevantes: fecha ('ds'), predicción puntual ('yhat'), y el intervalo de confianza
pronostico_ventas_30dias = forecast_ventas[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()

# Asegurarse de que las predicciones y los límites sean no negativos y redondear a entero
for col in ['yhat', 'yhat_lower', 'yhat_upper']:
     pronostico_ventas_30dias[col] = pronostico_ventas_30dias[col].apply(lambda x: max(0, round(x)))

# Mostrar la tabla completa del pronóstico
print(pronostico_ventas_30dias.to_string())


# --- 7. Visualizaciones (Opcional) ---
print("\n--- Visualizando Pronóstico ---")

# Gráfico del pronóstico (muestra datos históricos, predicción y intervalo de confianza)
fig_ventas = model_ventas.plot(forecast_ventas)
plt.title('Pronóstico de Ventas Totales Diarias (Tienda Trujillo)')
plt.xlabel('Fecha')
plt.ylabel('Ventas (Soles)')
plt.show()

# Gráfico de componentes (muestra tendencia, estacionalidad semanal y anual aprendidas por Prophet)
fig_comp = model_ventas.plot_components(forecast_ventas)
plt.show()